In [13]:
import os
import glob
import json
import pickle
import numpy as np
import pandas as pd
import datasetdatabase as dsdb

print(dsdb.version.VERSION)

os.environ["DSDB_PROCESS_LIMIT"] = "16"

1.0.31.stable


Reading the config file

In [14]:
with open("../qcb-regular-config.json", "r") as fjson:
    config_json = json.load(fjson)
print(json.dumps(config_json, indent=2))

{
  "meta": "QCB_cell_meta",
  "id": "cell_id",
  "cell_info": "/allen/aics/assay-dev/Analysis/QCB_database/cell_info",
  "raw_prefix": "raw.ome.tif",
  "seg_prefix": "seg.ome.tif",
  "mem_channel": 1,
  "dna_channel": 0,
  "str_channel": 2,
  "run": [
    {
      "structure_name": "cell",
      "status": "off",
      "channel": 1,
      "info": {
        "mode": "icc",
        "prefix": "cell",
        "shape": {
          "default": 0
        },
        "roundness": {
          "default": 0
        },
        "position": {
          "default": 0
        },
        "intensity": {
          "default": 0
        }
      },
      "description": "Cell features extracted from QCB images",
      "save_as": "QCB_CELL_feature.pkl"
    },
    {
      "structure_name": "dna",
      "status": "off",
      "channel": 0,
      "info": {
        "mode": "icc",
        "prefix": "dna",
        "shape": {
          "default": 0
        },
        "roundness": {
          "default": 0
        },
     

Loading metadata

In [15]:
with open(os.path.join("../../data-raw/",config_json["meta"]+".pkl"), "rb") as fp:
    df_meta = pickle.load(fp)
df_meta.head()

,cell_id,cell_ver,czi_filename,idx_in_stack,mitosis,mode,roi,src_csv,src_csv_row,str_ver,structure_name
0,c17a53d0-7ff6-4106-ab53-d2be3240f168,0.1.0,20161209_C01_002.czi,3,0,n,"[19, 195, 332, 513, 65, 295]",//allen/aics/assay-dev/StashSpreadsheets/assay...,7,0.1.0,FBL
1,77f11f0d-66d9-49c5-b2b8-be40777780ed,0.1.0,20161209_C01_002.czi,4,0,n,"[19, 187, 264, 453, 226, 444]",//allen/aics/assay-dev/StashSpreadsheets/assay...,8,0.1.0,FBL
2,427284a7-02b2-440b-a1b0-28c0b8f12379,0.1.0,20161209_C01_002.czi,5,0,n,"[19, 190, 73, 322, 422, 613]",//allen/aics/assay-dev/StashSpreadsheets/assay...,9,0.1.0,FBL
3,0241af5b-4133-4525-9d14-9a65f8a93f6f,0.1.0,20161209_C01_002.czi,7,0,n,"[19, 195, 355, 594, 68, 397]",//allen/aics/assay-dev/StashSpreadsheets/assay...,11,0.1.0,FBL
4,a705b03f-7af4-4fba-9e8f-8b20b085a1ab,0.1.0,20161209_C01_003.czi,1,0,n,"[8, 189, 78, 297, 228, 481]",//allen/aics/assay-dev/StashSpreadsheets/assay...,15,0.1.0,FBL


In [16]:
df_meta.shape

(10116, 11)

Connecting to database

In [17]:
prod = dsdb.DatasetDatabase(config=config_json["database"])

Uploading datasets

In [18]:
for struct in config_json["run"]:
    
    if struct["status"] == "on":
        
        ds_name = struct["save_as"].replace(".pkl","")
        
        print("running:", struct["structure_name"], "save as", ds_name)

        try:
            prod._purge_dataset(name=ds_name)
            print("\t dataset",ds_name,"purged")
        except: pass
        
        with open(os.path.join("../../data-raw/",struct["save_as"]), "rb") as fp:
            df = pickle.load(fp)
  
        df = df.reset_index(drop=False)
        df = df.rename(index=str, columns={"index":"cell_id"})
        ds = dsdb.Dataset(df, name=ds_name, description=struct["description"])

        ds.upload_to(prod)

running: FBL save as QCB_FBL_feature
	 dataset QCB_FBL_feature purged
Storing algorithm parameters.
Input dataset already exists in database. 1
Tearing down object...
[============================================================] 100.0% 1166/1166 ~ 0:0:0 remaining
